In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install fairseq
!pip install fastBPE

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 20.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 14.1 MB/s eta 0:00:00
  Prepar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import re
import random
import seaborn as sns
from tqdm.notebook import tqdm
import torch
import argparse
import pickle
import numpy as np
from os.path import join
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.manifold import TSNE

from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from transformers.modeling_utils import *
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
from transformers import AutoModel, AutoTokenizer

import argparse

In [ ]:
warmup_sets = '/content/drive/MyDrive/pRoBERTa/data/converted_data_warmup_tocken.json'
train_sets = '/content/drive/MyDrive/pRoBERTa/data/converted_data_warmup_tocken.json'
dest = '/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection'
model = 'base'
epochs = 20
batch_size_gpu = 4
batch_size_accumulated = 256
lr_base = 1e-5
lr_linear = 1e-3

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')

Using device "cuda"


# Load data

In [ ]:
import random
class LoadDataFromJson(Dataset):
    def __init__(self, datasets):
        self.samples = []
        with open(datasets) as file:
            data = json.load(file)

        for id, item in data.items():
            sentences = item["context"]
            if item["evidence"] != None:
              for sentence in sentences:
                evidence = item['evidence'] in sentence
                if evidence:
                  data_sample = {}
                  data_sample['id'] = id
                  data_sample['claim'] = item['claim']
                  data_sample['sentence'] = sentence
                  data_sample['evidence'] = True
                  data_sample['domain'] = item['domain']
                  self.samples.append(data_sample)
            else:
              random_sentence = random.sample(sentences, 2)
              for sentence in random_sentence:
                data_sample = {}
                data_sample['id'] = id
                data_sample['claim'] = item['claim']
                data_sample['sentence'] = sentence
                data_sample['evidence'] = False
                data_sample['domain'] = item['domain']
                self.samples.append(data_sample)

    def to_dataframe(self):
        return pd.DataFrame(self.samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [ ]:
data_train = LoadDataFromJson(train_sets)
df_train = data_train.to_dataframe()

In [ ]:
df_train

,id,claim,sentence,evidence,domain
0,36112,trong năm nay hai địa_phương dẫn_đầu và gần nh...,năm nay hai địa_phương có lượng hồ_sơ dẫn_đầu ...,True,khoahoc
1,27839,faa có hơn 10 nhân_viên,ngoài cuộc điều_tra của faa spacex cũng phải đ...,False,khoahoc
2,27839,faa có hơn 10 nhân_viên,cục hàng_không liên_bang mỹ ( faa ) đang giám_...,False,khoahoc
3,36633,do_đó chị đã quyết_định mở hàng ốc nhồi hơn mộ...,vì_vậy sau khi trở_lại sài_gòn chị đã quyết_đị...,True,dulich
4,18939,số người mắc_chứng tâm_lý ptsd tăng mạnh sau m...,hai người trong tiểu_đoàn của anh đã tự_sát từ...,False,thegioi
...,...,...,...,...,...
1457,10840,gueorguiev được phép trò_chuyện với người khác...,gueorguiev được phép trò_chuyện với mọi người ...,True,thegioi
1458,15809,tỉnh gia_lai đã và đang huy_động người_dân cân...,gia_đình ông bỏ khoảng 200 triệu đồng đầu_tư 4...,False,thoisu
1459,15809,tỉnh gia_lai đã và đang huy_động người_dân cân...,năm 202023 toàn tỉnh gia_lai có trên 30 ha mía,False,thoisu
1460,881,em rất mơ_hồ về các mức lương trong ngành kinh_tế,em rất mơ_hồ về các ngành,False,giaoduc


In [ ]:
df_train['evidence'].value_counts()

False    936
True     526
Name: evidence, dtype: int64

In [ ]:
# filtered_df = df_train.loc[(df_train['sentence'].apply(lambda x: len(x.split())) < 5) & (df_train['evidence'] == False)]
# filtered_df
# # Xóa những dòng có chiều dài sentence < 5 và evidence = 0
# df_train = df_train.drop(df_train[df_train.isin(filtered_df.to_dict('list')).all(1)].index)

In [ ]:
# # Xóa những dòng có chiều dài sentence < 5 và evidence = 0
# df_train = df_train.drop(df_train[df_train.isin(filtered_df.to_dict('list')).all(1)].index)

In [ ]:
df_train = df_train.sample(frac=1, random_state=42)

In [ ]:
df_train

,id,claim,sentence,evidence,domain
892,46877,đây là nơi thu_hút người không có tài_năng khô...,đây là nơi thu_hút người tài có năng_lực hấp_t...,True,khoahoc
1106,6561,vị ngọt từ hoa_quả có lợi hơn so với hóa học,người_mẫu chia_sẻ việc bôi kem chống nắng trán...,False,suckhoe
413,10577,ngày 16/1 khi công_bố kết_quả khám_nghiệm con ...,kết_quả khám_nghiệm được công_bố hôm 16/1 cho ...,True,khoahoc
522,10515,hành_vi ăn thịt đồng_loại của cá_sấu dao_động ...,tuy_nhiên tần_suất xảy ra hành_vi này dao_động...,True,khoahoc
1036,47315,một điều thú_vị khác là những họ_hàng gần nhất...,một điều thú_vị khác là những họ_hàng gần nhất...,True,khoahoc
...,...,...,...,...,...
1130,20038,tuyến phà gót cái viềng vẫn giữ giá vé không đ...,nếu đi từ tp hải_phòng người_dân và du_khách c...,False,dulich
1294,12395,kiều_bào ở cả châu_âu và nhật_bản đều có nhu_c...,trong chuyến làm_việc tới châu_âu mới_đây ông ...,True,thegioi
860,40689,sinh_viên không bị bảo_tàng leeum yêu_cầu phải...,bảo_tàng leeum quyết_định không yêu_cầu sinh_v...,True,thegioi
1459,15809,tỉnh gia_lai đã và đang huy_động người_dân cân...,năm 202023 toàn tỉnh gia_lai có trên 30 ha mía,False,thoisu


In [ ]:
w = df_train.loc[df_train['id'] == "46576"].sentence
w

Series([], Name: sentence, dtype: object)

In [ ]:
a = df_train[df_train['evidence'] == True]
a

,id,claim,sentence,evidence,domain
892,46877,đây là nơi thu_hút người không có tài_năng khô...,đây là nơi thu_hút người tài có năng_lực hấp_t...,True,khoahoc
413,10577,ngày 16/1 khi công_bố kết_quả khám_nghiệm con ...,kết_quả khám_nghiệm được công_bố hôm 16/1 cho ...,True,khoahoc
522,10515,hành_vi ăn thịt đồng_loại của cá_sấu dao_động ...,tuy_nhiên tần_suất xảy ra hành_vi này dao_động...,True,khoahoc
1036,47315,một điều thú_vị khác là những họ_hàng gần nhất...,một điều thú_vị khác là những họ_hàng gần nhất...,True,khoahoc
1161,11066,nam_giới cần tiêm xuất_tinh tối_đa 7 ngày trướ...,trước khi làm xét_nghiệm nam_giới cần kiêng xu...,True,suckhoe
...,...,...,...,...,...
1123,13992,người nông_dân dễ_dàng tìm_kiếm các loại thuốc...,vì_vậy phần_mềm_tra cứu thuốc trên điện_thoại ...,True,thoisu
330,7210,ngoài việc khám_phá vùng_đất mới để học cách s...,song_song quá_trình cùng học_tập rèn_luyện kỹ_...,True,giaoduc
1294,12395,kiều_bào ở cả châu_âu và nhật_bản đều có nhu_c...,trong chuyến làm_việc tới châu_âu mới_đây ông ...,True,thegioi
860,40689,sinh_viên không bị bảo_tàng leeum yêu_cầu phải...,bảo_tàng leeum quyết_định không yêu_cầu sinh_v...,True,thegioi


In [ ]:
# data_val = LoadDataFromJson(warmup_sets)
# df_val = data_val.to_dataframe()
# df_val = df_val.sample(frac=1, random_state=42)

In [ ]:
# df_val

In [ ]:
# df_train.to_json('/content/drive/MyDrive/pRoBERTa/data/df_train_rationale_selection.json')

In [ ]:
df_train = df_train.head(100)

# Chuẩn bị dữ liệu train

In [ ]:
# load model phoBert và tokenizer của model đó
model_bert = AutoModel.from_pretrained(f'vinai/phobert-{model}').to(device).eval()
tokenizer = AutoTokenizer.from_pretrained(f"vinai/phobert-{model}", use_fast=False)

In [ ]:
# Load BPE encoder
parser = argparse.ArgumentParser()
parser.add_argument(
    '--bpe-codes',
    default=f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/bpe.codes',
    required=False,
    type=str,
    help='path to fastBPE BPE'
)

args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file(f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/dict.txt')

In [ ]:
input_sent = df_train.loc[:, ['claim','sentence']]
input_labels = df_train['evidence']

In [ ]:
# encode label
le = LabelEncoder()
train_labels = le.fit_transform(input_labels)
print(le.classes_)

[False  True]


In [ ]:
from sklearn.model_selection import train_test_split
train_sents, val_sents, train_labels, val_labels = train_test_split(input_sent, train_labels, test_size=0.1, random_state=42)

In [ ]:
# val_sents = df_val.loc[:, ['claim','sentence']]
# val_labels = df_val['evidence']

In [ ]:
# # encode label
# le = LabelEncoder()
# train_labels = le.fit_transform(train_labels)
# print(le.classes_)
# val_labels = le.fit_transform(val_labels)
# print(le.classes_)

In [ ]:
# Thực hiện chuyển dataset ban đầu thành subword index
MAX_LEN = 256

train_ids = []
print('Processing train set ...')
for index, row in tqdm(train_sents.iterrows()):
  claim_value = row['claim']
  sentence_value = row['sentence']
  subwords = '[CLS]' + bpe.encode(claim_value) + '[SEP]' + bpe.encode(sentence_value) + '[SEP]'
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  train_ids.append(encoded_sent)

val_ids = []
print('Processing test set ...')
for index, row in tqdm(val_sents.iterrows()):
  claim_value = row['claim']
  sentence_value = row['sentence']
  subwords = '[CLS]' + bpe.encode(claim_value) + '[SEP]' + bpe.encode(sentence_value) + '[SEP]'
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long()
  val_ids.append(encoded_sent)

Processing train set ...


0it [00:00, ?it/s]

Processing test set ...


0it [00:00, ?it/s]

In [ ]:
# thêm padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype='long', value=1, truncating='post', padding='post')


In [ ]:
# Tạo mask để BERT không chú ý tới padding (pad id = 1)
train_masks = []
for sent in train_ids:
  mask = [int(token_id != 1) for token_id in sent]
  train_masks.append(mask)

val_masks = []
for sent in val_ids:
  mask = [int(token_id != 1) for token_id in sent]
  val_masks.append(mask)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 16

# convert sang tensor
train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
# tạo data loader để train
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)


# Xây dựng model

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

NUM_CLASSES = 2

# load file config
config = RobertaConfig.from_pretrained(
    f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/config.json',
    from_tf=False,
    num_labels=NUM_CLASSES,
    output_hidden_states=False
)

# load model
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    f'/content/drive/MyDrive/pRoBERTa/PhoBERT_{model}_transformers/model.bin',
    config=config
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/pRoBERTa/PhoBERT_base_transformers/model.bin and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
BERT_SA

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=0)
      (position_embeddings): Embedding(258, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaModel

class CustomRobertaClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super(CustomRobertaClassificationHead, self).__init__()
        self.dense = nn.Linear(768, 768)
        self.dropout = nn.Dropout(p=0.1)
        self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=256, batch_first=True)
        self.fc1 = nn.Linear(256, 128)
        self.out_proj = nn.Linear(128, num_labels)

    def forward(self, hidden_states):
        x = self.dense(hidden_states)  # Ánh xạ hidden_states qua lớp tuyến tính dense
        x = torch.relu(x)  # Áp dụng hàm kích hoạt ReLU
        x = self.dropout(x)  # Áp dụng dropout
        _, (h_n, _) = self.lstm(x)  # Đưa x qua LSTM và lấy hidden state cuối cùng
        x = self.fc1(h_n[-1])
        x = self.out_proj(x)

        return x

class CustomRobertaForSequenceClassification(nn.Module):
    def __init__(self, model, num_labels):
        super(CustomRobertaForSequenceClassification, self).__init__()

        self.roberta = model
        self.roberta.classifier = CustomRobertaClassificationHead(hidden_size=768, num_labels=num_labels)

    def forward(self, input_ids = None, token_type_ids = None, attention_mask = None, labels = None):
        outputs = self.roberta(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)
        return outputs

    def save_pretrained(self, PATH):
       # Create the directory if it doesn't exist
      os.makedirs(PATH, exist_ok=True)

      # Save the model's state_dict to a file
      model_path = os.path.join(PATH, 'model.bin')
      torch.save(self.state_dict(), model_path)


In [ ]:
custom_model = CustomRobertaForSequenceClassification(BERT_SA, 2)

In [ ]:
# Hàm đánh giá độ chính xác
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  F1_score = f1_score(pred_flat, labels_flat, average='macro')

  return accuracy_score(pred_flat, labels_flat), F1_score

# Train model

In [ ]:
custom_model.cuda()

CustomRobertaForSequenceClassification(
  (roberta): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(64001, 768, padding_idx=0)
        (position_embeddings): Embedding(258, 768, padding_idx=0)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput

In [ ]:
# định nghĩa tham số
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 15
save_dir = '/content/drive/MyDrive'
best_acc = 0

# lưu lại lịch sử
losses = []
train_accs = []
val_accs = []

# define optimizer
param_optimizer = list(custom_model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)

# Train theo từng epoch
for epoch_i in range(0, epochs):
  print('================= Epoch {} / {} ====================='.format(epoch_i + 1, epochs))
  print('Training ... ')

  # Tham số train
  total_loss = 0
  custom_model.train()
  train_accuracy = 0
  nb_train_steps = 0
  train_f1 = 0

  # Train từng batch
  for step, batch in enumerate(tqdm(train_dataloader)):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    # check======
    print("step: ", step)
    print("batch: " ,batch)
    print("inputids" ,b_input_ids)
    print("inputids_shape" ,b_input_ids.shape)

    print("mask:  ", b_input_mask)
    print("mask_shape:  ", b_input_mask.shape)

    print("labels: ", b_labels)
    print("labels: ", b_labels.shape)
    #======

    custom_model.zero_grad()
    outputs = custom_model(
        b_input_ids,
        token_type_ids=None,
        attention_mask=b_input_mask,
        labels=b_labels
    )

    #check====
    print("outputs: " ,outputs)
    print("outputs_shape: " ,outputs[1].shape)

    #======

    loss = outputs[0]
    total_loss += loss.item()

    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
    train_accuracy += tmp_train_accuracy
    train_f1 += tmp_train_f1
    nb_train_steps += 1

    loss.backward()
    torch.nn.utils.clip_grad_norm_(custom_model.parameters(), 1.0)
    optimizer.step()

  # tính average loss và accuracy cho epoch đó
  avg_train_loss = total_loss / len(train_dataloader)
  losses.append(avg_train_loss)
  train_accs.append(train_accuracy/nb_train_steps)

  print('Accuracy: {0:.4f}'.format(train_accuracy/nb_train_steps))
  print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
  print(" Average training loss: {0:.4f}".format(avg_train_loss))

  print("Running Validation...")

  custom_model.eval()

  # tham số validate
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  eval_f1 = 0

  # đánh giá
  for step, batch in enumerate(tqdm(val_dataloader)):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    custom_model.zero_grad()
    outputs = custom_model(
        b_input_ids,
        token_type_ids=None,
        attention_mask=b_input_mask,
        labels=b_labels
    )
    loss = outputs[0]
    total_loss += loss.item()

    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    eval_f1 += tmp_eval_f1
    nb_eval_steps += 1


  avg_val_acc = eval_accuracy/nb_eval_steps
  val_accs.append(avg_val_acc)
  print(" Accuracy: {0:.4f}".format(avg_val_acc))
  print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))

  # save checkpoint
  if avg_val_acc > best_acc:
    # save checkpoint
    print('Saved model to %s' %save_dir)
    custom_model.save_pretrained(save_dir)

    best_acc = avg_val_acc

  # early stopping
  # if epoch_i >= 4: # Khởi động 3 epochs
  #   if avg_val_acc < val_accs[-2]:
  #     # bị overfit
  #     print('Early stopping!')
  #     break
print("Training complete!")

================= Epoch 1 / 15 =====================
Training ... 


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/6 [00:00<?, ?it/s]

step:  0
batch:  [tensor([[    3,   292,   915,  ...,     1,     1,     1],
        [    3, 36157, 22143,  ...,     1,     1,     1],
        [    3,  2324,  1124,  ...,     1,     1,     1],
        ...,
        [    3, 11759,   692,  ...,     1,     1,     1],
        [    3,  2479,  4104,  ...,     1,     1,     1],
        [    3,  1910,   311,  ...,     1,     1,     1]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])]
inputids tensor([[    3,   292,   915,  ...,     1,     1,     1],
        [    3, 36157, 22143,  ...,     1,     1,     1],
        [    3,  2324,  1124,  ...,     1,     1,     1],
        ...,
        [    3, 11759,   692,  ...,     1,     1,     1],
        [    3,  2479,  4104,  ...,     1,     1,     1],
        [    3,  1910,   311, 

  0%|          | 0/1 [00:00<?, ?it/s]

...

 Accuracy: 0.8000
 F1 score: 0.7619
================= Epoch 15 / 15 =====================
Training ... 


  0%|          | 0/6 [00:00<?, ?it/s]

step:  0
batch:  [tensor([[    3,   292,   915,  ...,     1,     1,     1],
        [    3, 36157, 22143,  ...,     1,     1,     1],
        [    3,  2324,  1124,  ...,     1,     1,     1],
        ...,
        [    3, 11759,   692,  ...,     1,     1,     1],
        [    3,  2479,  4104,  ...,     1,     1,     1],
        [    3,  1910,   311,  ...,     1,     1,     1]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])]
inputids tensor([[    3,   292,   915,  ...,     1,     1,     1],
        [    3, 36157, 22143,  ...,     1,     1,     1],
        [    3,  2324,  1124,  ...,     1,     1,     1],
        ...,
        [    3, 11759,   692,  ...,     1,     1,     1],
        [    3,  2479,  4104,  ...,     1,     1,     1],
        [    3,  1910,   311, 

  0%|          | 0/1 [00:00<?, ?it/s]

 Accuracy: 0.8000
 F1 score: 0.7619
Training complete!


In [ ]:
# save_dir = '/content/drive/MyDrive/pRoBERTa/trained_module/rationale_selection'
# custom_model.save_pretrained(save_dir)